# SQLite Database Prototype

> **⚠️ PORTFOLIO/DEMO CODE**  
> This notebook is preserved for portfolio purposes and uses `demo_prototype.db` instead of the production database.  
> The code is functional but isolated from the production `sku_catalog.db` file.

This notebook designs and prototypes a SQLite database for the SKU generation system, with the intent to migrate to PostgreSQL on a Synology NAS in the future.

## Design Principles
- Use SQLAlchemy ORM for database-agnostic code
- Design schema to support future PostgreSQL migration
- Maintain compatibility with existing Numbers-based workflow during transition

## Phase 1: Schema Design

### Current Numbers File Structure (Reference)

From `catalog.numbers`:
- **Product_Catalog sheet**: Main product models with metadata
- **Feature sheets**: Colors, Sizes, Designs, Materials, etc.
- Each product has multiple features that create permutations

### Proposed Database Schema

Below we'll design each table structure before implementation.

## Phase 2: SQLite Implementation

Now let's build the actual database using SQLAlchemy ORM.

### Step 1: Import Dependencies

We'll use SQLAlchemy for database-agnostic ORM code.

In [1]:
from sqlalchemy import create_engine, Column, Integer, String, Text, Numeric, Boolean, DateTime, ForeignKey, UniqueConstraint, Index
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship
from datetime import datetime
import os

print("SQLAlchemy imported successfully!")

SQLAlchemy imported successfully!


### Step 2: Define Database Models

These classes represent our 7 tables as SQLAlchemy ORM models.

In [2]:
# Create base class for all models
Base = declarative_base()

class Category(Base):
    __tablename__ = 'categories'
    
    id = Column(Integer, primary_key=True, autoincrement=True)
    category = Column(String(100), nullable=False)
    subcategory = Column(String(100))
    created_at = Column(DateTime, default=datetime.utcnow)
    updated_at = Column(DateTime, default=datetime.utcnow, onupdate=datetime.utcnow)
    
    # Relationships
    products = relationship("Product", back_populates="category")
    
    # Indexes
    __table_args__ = (
        Index('idx_category_subcategory', 'category', 'subcategory'),
    )
    
    def __repr__(self):
        return f"<Category(id={self.id}, category='{self.category}', subcategory='{self.subcategory}')>"


class Product(Base):
    __tablename__ = 'products'
    
    id = Column(Integer, primary_key=True, autoincrement=True)
    product_name = Column(String(200), nullable=False, unique=True)
    category_id = Column(Integer, ForeignKey('categories.id'))
    description = Column(Text)
    base_price = Column(Numeric(10, 2))
    status = Column(String(20), default='active')
    created_at = Column(DateTime, default=datetime.utcnow)
    updated_at = Column(DateTime, default=datetime.utcnow, onupdate=datetime.utcnow)
    
    # Relationships
    category = relationship("Category", back_populates="products")
    product_features = relationship("ProductFeature", back_populates="product")
    skus = relationship("SKU", back_populates="product")
    
    # Indexes
    __table_args__ = (
        Index('idx_product_name', 'product_name'),
        Index('idx_category_id', 'category_id'),
        Index('idx_status', 'status'),
    )
    
    def __repr__(self):
        return f"<Product(id={self.id}, name='{self.product_name}', status='{self.status}')>"


class FeatureType(Base):
    __tablename__ = 'feature_types'
    
    id = Column(Integer, primary_key=True, autoincrement=True)
    feature_type = Column(String(50), nullable=False, unique=True)
    display_order = Column(Integer)
    is_required = Column(Boolean, default=False)
    created_at = Column(DateTime, default=datetime.utcnow)
    
    # Relationships
    features = relationship("Feature", back_populates="feature_type")
    
    def __repr__(self):
        return f"<FeatureType(id={self.id}, type='{self.feature_type}', required={self.is_required})>"


class Feature(Base):
    __tablename__ = 'features'
    
    id = Column(Integer, primary_key=True, autoincrement=True)
    feature_type_id = Column(Integer, ForeignKey('feature_types.id'))
    code = Column(String(20), nullable=False)
    name = Column(String(100), nullable=False)
    description = Column(Text)
    sort_order = Column(Integer)
    is_active = Column(Boolean, default=True)
    created_at = Column(DateTime, default=datetime.utcnow)
    updated_at = Column(DateTime, default=datetime.utcnow, onupdate=datetime.utcnow)
    
    # Relationships
    feature_type = relationship("FeatureType", back_populates="features")
    product_features = relationship("ProductFeature", back_populates="feature")
    sku_features = relationship("SKUFeature", back_populates="feature")
    
    # Constraints and Indexes
    __table_args__ = (
        UniqueConstraint('feature_type_id', 'code', name='uq_feature_type_code'),
        Index('idx_feature_type_code', 'feature_type_id', 'code'),
        Index('idx_is_active', 'is_active'),
    )
    
    def __repr__(self):
        return f"<Feature(id={self.id}, code='{self.code}', name='{self.name}')>"


class ProductFeature(Base):
    __tablename__ = 'product_features'
    
    id = Column(Integer, primary_key=True, autoincrement=True)
    product_id = Column(Integer, ForeignKey('products.id'))
    feature_id = Column(Integer, ForeignKey('features.id'))
    created_at = Column(DateTime, default=datetime.utcnow)
    
    # Relationships
    product = relationship("Product", back_populates="product_features")
    feature = relationship("Feature", back_populates="product_features")
    
    # Constraints and Indexes
    __table_args__ = (
        UniqueConstraint('product_id', 'feature_id', name='uq_product_feature'),
        Index('idx_product_id', 'product_id'),
        Index('idx_feature_id', 'feature_id'),
    )
    
    def __repr__(self):
        return f"<ProductFeature(product_id={self.product_id}, feature_id={self.feature_id})>"


class SKU(Base):
    __tablename__ = 'skus'
    
    id = Column(Integer, primary_key=True, autoincrement=True)
    sku = Column(String(100), nullable=False, unique=True)
    product_id = Column(Integer, ForeignKey('products.id'))
    batch_name = Column(String(200))
    price = Column(Numeric(10, 2))
    inventory_qty = Column(Integer, default=0)
    status = Column(String(20), default='active')
    generated_at = Column(DateTime, default=datetime.utcnow)
    updated_at = Column(DateTime, default=datetime.utcnow, onupdate=datetime.utcnow)
    
    # Relationships
    product = relationship("Product", back_populates="skus")
    sku_features = relationship("SKUFeature", back_populates="sku")
    
    # Indexes
    __table_args__ = (
        Index('idx_sku', 'sku'),
        Index('idx_product_id_sku', 'product_id'),
        Index('idx_batch_name', 'batch_name'),
        Index('idx_status_sku', 'status'),
    )
    
    def __repr__(self):
        return f"<SKU(id={self.id}, sku='{self.sku}', status='{self.status}')>"


class SKUFeature(Base):
    __tablename__ = 'sku_features'
    
    id = Column(Integer, primary_key=True, autoincrement=True)
    sku_id = Column(Integer, ForeignKey('skus.id'))
    feature_id = Column(Integer, ForeignKey('features.id'))
    created_at = Column(DateTime, default=datetime.utcnow)
    
    # Relationships
    sku = relationship("SKU", back_populates="sku_features")
    feature = relationship("Feature", back_populates="sku_features")
    
    # Constraints and Indexes
    __table_args__ = (
        UniqueConstraint('sku_id', 'feature_id', name='uq_sku_feature'),
        Index('idx_sku_id', 'sku_id'),
        Index('idx_feature_id_sku', 'feature_id'),
    )
    
    def __repr__(self):
        return f"<SKUFeature(sku_id={self.sku_id}, feature_id={self.feature_id})>"

print("✓ All 7 database models defined successfully!")

✓ All 7 database models defined successfully!


/var/folders/qt/34m6wpbn699_75xmv603443m0000gn/T/ipykernel_55871/122878795.py:2: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


### Step 3: Initialize the Database

Create the SQLite database file and all tables.

In [ ]:
# ⚠️ DEMO/PORTFOLIO CODE ONLY - Uses separate demo database
# Set database path to demo file (won't affect production sku_catalog.db)
DB_PATH = 'demo_prototype.db'

# Create engine (this is the connection to the database)
engine = create_engine(f'sqlite:///{DB_PATH}', echo=True)

# Create all tables defined in Base
Base.metadata.create_all(engine)

# Create session factory
Session = sessionmaker(bind=engine)

# Verify database was created
if os.path.exists(DB_PATH):
    db_size = os.path.getsize(DB_PATH)
    print(f"\n✓ Database created successfully!")
    print(f"  Location: {os.path.abspath(DB_PATH)}")
    print(f"  Size: {db_size} bytes")
    print(f"  Tables created: {len(Base.metadata.tables)}")
    print(f"  Table names: {list(Base.metadata.tables.keys())}")
else:
    print("✗ Database creation failed!")

2025-12-02 14:40:11,379 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-02 14:40:11,380 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("categories")
2025-12-02 14:40:11,380 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-12-02 14:40:11,381 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("categories")
2025-12-02 14:40:11,381 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-12-02 14:40:11,381 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("products")
2025-12-02 14:40:11,381 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-12-02 14:40:11,382 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("products")
2025-12-02 14:40:11,382 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-12-02 14:40:11,382 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("feature_types")
2025-12-02 14:40:11,383 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-12-02 14:40:11,383 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("feature_types")
2025-12-02 14:40:11,383 INFO sqlalchemy.engine

### Step 4: Verify Database Schema

Let's inspect the database structure to confirm everything was created correctly.

In [4]:
import sqlite3

# Connect to the database to inspect schema
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

# Get list of all tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

print("Database Tables:")
print("=" * 60)
for table in tables:
    table_name = table[0]
    print(f"\n{table_name.upper()}")
    print("-" * 60)
    
    # Get columns for this table
    cursor.execute(f"PRAGMA table_info({table_name});")
    columns = cursor.fetchall()
    
    for col in columns:
        col_id, name, col_type, not_null, default, pk = col
        pk_marker = " [PK]" if pk else ""
        null_marker = " NOT NULL" if not_null else ""
        default_marker = f" DEFAULT {default}" if default else ""
        print(f"  {name:<20} {col_type:<15}{pk_marker}{null_marker}{default_marker}")
    
    # Get indexes for this table
    cursor.execute(f"PRAGMA index_list({table_name});")
    indexes = cursor.fetchall()
    if indexes:
        print(f"\n  Indexes:")
        for idx in indexes:
            idx_name = idx[1]
            print(f"    - {idx_name}")

conn.close()
print("\n" + "=" * 60)
print("✓ Schema verification complete!")

Database Tables:

CATEGORIES
------------------------------------------------------------
  id                   INTEGER         [PK] NOT NULL
  category             VARCHAR(100)    NOT NULL
  subcategory          VARCHAR(100)   
  created_at           DATETIME       
  updated_at           DATETIME       

  Indexes:
    - idx_category_subcategory

FEATURE_TYPES
------------------------------------------------------------
  id                   INTEGER         [PK] NOT NULL
  feature_type         VARCHAR(50)     NOT NULL
  display_order        INTEGER        
  is_required          BOOLEAN        
  created_at           DATETIME       

  Indexes:
    - sqlite_autoindex_feature_types_1

PRODUCTS
------------------------------------------------------------
  id                   INTEGER         [PK] NOT NULL
  product_name         VARCHAR(200)    NOT NULL
  category_id          INTEGER        
  description          TEXT           
  base_price           NUMERIC(10, 2) 
  status       

### Step 5: Test with Sample Data

Let's insert some test data to make sure everything works.

In [5]:
# Create a new session
session = Session()

try:
    # Create a test category
    test_category = Category(
        category="Apparel",
        subcategory="T-Shirts"
    )
    session.add(test_category)
    session.flush()  # Get the ID without committing
    
    # Create a test product
    test_product = Product(
        product_name="Test Basic Tee",
        category_id=test_category.id,
        description="A test product for validation",
        base_price=19.99,
        status="active"
    )
    session.add(test_product)
    session.flush()
    
    # Create feature types
    color_type = FeatureType(feature_type="Color", display_order=1, is_required=True)
    size_type = FeatureType(feature_type="Size", display_order=2, is_required=True)
    session.add_all([color_type, size_type])
    session.flush()
    
    # Create some features
    black_color = Feature(
        feature_type_id=color_type.id,
        code="BLK",
        name="Black",
        sort_order=1,
        is_active=True
    )
    
    medium_size = Feature(
        feature_type_id=size_type.id,
        code="M",
        name="Medium",
        sort_order=1,
        is_active=True
    )
    session.add_all([black_color, medium_size])
    session.flush()
    
    # Link features to product
    prod_color = ProductFeature(product_id=test_product.id, feature_id=black_color.id)
    prod_size = ProductFeature(product_id=test_product.id, feature_id=medium_size.id)
    session.add_all([prod_color, prod_size])
    session.flush()
    
    # Create a test SKU
    test_sku = SKU(
        sku="TEST-M-BLK",
        product_id=test_product.id,
        batch_name="Test Batch",
        price=19.99,
        inventory_qty=100,
        status="active"
    )
    session.add(test_sku)
    session.flush()
    
    # Link features to SKU
    sku_color = SKUFeature(sku_id=test_sku.id, feature_id=black_color.id)
    sku_size = SKUFeature(sku_id=test_sku.id, feature_id=medium_size.id)
    session.add_all([sku_color, sku_size])
    
    # Commit all changes
    session.commit()
    
    print("✓ Test data inserted successfully!")
    print(f"\n  Category: {test_category}")
    print(f"  Product: {test_product}")
    print(f"  Feature Types: {color_type}, {size_type}")
    print(f"  Features: {black_color}, {medium_size}")
    print(f"  SKU: {test_sku}")
    print(f"\n  Total records created: 10")
    
except Exception as e:
    session.rollback()
    print(f"✗ Error inserting test data: {e}")
    raise
finally:
    session.close()

2025-12-02 14:42:01,334 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-02 14:42:01,335 INFO sqlalchemy.engine.Engine INSERT INTO categories (category, subcategory, created_at, updated_at) VALUES (?, ?, ?, ?)
2025-12-02 14:42:01,335 INFO sqlalchemy.engine.Engine [generated in 0.00032s] ('Apparel', 'T-Shirts', '2025-12-02 20:42:01.335589', '2025-12-02 20:42:01.335592')
2025-12-02 14:42:01,337 INFO sqlalchemy.engine.Engine INSERT INTO products (product_name, category_id, description, base_price, status, created_at, updated_at) VALUES (?, ?, ?, ?, ?, ?, ?)
2025-12-02 14:42:01,337 INFO sqlalchemy.engine.Engine [generated in 0.00046s] ('Test Basic Tee', 1, 'A test product for validation', 19.99, 'active', '2025-12-02 20:42:01.337253', '2025-12-02 20:42:01.337255')
2025-12-02 14:42:01,339 INFO sqlalchemy.engine.Engine INSERT INTO feature_types (feature_type, display_order, is_required, created_at) VALUES (?, ?, ?, ?) RETURNING id
2025-12-02 14:42:01,339 INFO sqlalchemy.engine.Engine [

### Step 6: Query Test Data

Verify we can retrieve the data we just inserted.

In [6]:
# Create a new session for querying
session = Session()

try:
    # Query all SKUs with their related data
    skus = session.query(SKU).all()
    
    print("SKUs in Database:")
    print("=" * 80)
    
    for sku in skus:
        print(f"\nSKU: {sku.sku}")
        print(f"  Product: {sku.product.product_name}")
        print(f"  Category: {sku.product.category.category} → {sku.product.category.subcategory}")
        print(f"  Price: ${sku.price}")
        print(f"  Inventory: {sku.inventory_qty}")
        print(f"  Status: {sku.status}")
        
        # Get features for this SKU
        print(f"  Features:")
        for sku_feature in sku.sku_features:
            feature = sku_feature.feature
            print(f"    - {feature.feature_type.feature_type}: {feature.name} ({feature.code})")
    
    print("\n" + "=" * 80)
    print(f"✓ Query successful! Found {len(skus)} SKU(s)")
    
except Exception as e:
    print(f"✗ Error querying data: {e}")
    raise
finally:
    session.close()

2025-12-02 14:42:07,428 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-02 14:42:07,429 INFO sqlalchemy.engine.Engine SELECT skus.id AS skus_id, skus.sku AS skus_sku, skus.product_id AS skus_product_id, skus.batch_name AS skus_batch_name, skus.price AS skus_price, skus.inventory_qty AS skus_inventory_qty, skus.status AS skus_status, skus.generated_at AS skus_generated_at, skus.updated_at AS skus_updated_at 
FROM skus
2025-12-02 14:42:07,429 INFO sqlalchemy.engine.Engine [generated in 0.00042s] ()
SKUs in Database:

SKU: TEST-M-BLK
2025-12-02 14:42:07,432 INFO sqlalchemy.engine.Engine SELECT products.id AS products_id, products.product_name AS products_product_name, products.category_id AS products_category_id, products.description AS products_description, products.base_price AS products_base_price, products.status AS products_status, products.created_at AS products_created_at, products.updated_at AS products_updated_at 
FROM products 
WHERE products.id = ?
2025-12-02 14:42:07,43

## Phase 3: Data Migration from Numbers File

Now let's migrate your existing catalog data from the Numbers file into the database.

### Step 1: Load Numbers File

Reuse the data loading function from sku-gen-beta.

In [ ]:
import pandas as pd
from numbers_parser import Document

def load_catalog_data(file_path):
    """Load Apple Numbers file into DataFrames."""
    doc = Document(file_path)
    sheets = doc.sheets
    tables = {}

    for sheet in sheets:
        for table in sheet.tables:
            table_name = table.name.replace(" ", "_")
            data = table.rows(values_only=True)
            tables[table_name] = pd.DataFrame(data[1:], columns=data[0])

    # Clean column names for Product_Catalog
    tables['Product_Catalog'].columns = tables['Product_Catalog'].columns.str.replace(' ', '_')
    
    return tables

# Load the catalog
catalog_file = "../CHARTS/catalog.numbers"
catalog_tables = load_catalog_data(catalog_file)

print("✓ Catalog loaded successfully!")
print(f"  Tables: {list(catalog_tables.keys())}")
print(f"  Products: {len(catalog_tables['Product_Catalog'])}")
print(f"  Feature types: {len(catalog_tables.keys()) - 2}")  # Exclude Product_Catalog and Prefix_Codes

✓ Catalog loaded successfully!
  Tables: ['Product_Catalog', 'Size', 'Prefix_Codes', 'Main_Category', 'Sub_Category', 'Fit', 'Color', 'Design', 'Material', 'Scent']
  Products: 18
  Feature types: 8


### Clean Up Test Data

Remove the test data before migrating real catalog data.

In [7]:
session = Session()

try:
    # Delete in reverse order of dependencies
    deleted_counts = {}
    
    # SKU Features (depends on SKUs and Features)
    deleted_counts['SKU Features'] = session.query(SKUFeature).delete()
    
    # SKUs (depends on Products)
    deleted_counts['SKUs'] = session.query(SKU).delete()
    
    # Product Features (depends on Products and Features)
    deleted_counts['Product Features'] = session.query(ProductFeature).delete()
    
    # Features (depends on Feature Types)
    deleted_counts['Features'] = session.query(Feature).delete()
    
    # Feature Types (no dependencies)
    deleted_counts['Feature Types'] = session.query(FeatureType).delete()
    
    # Products (depends on Categories)
    deleted_counts['Products'] = session.query(Product).delete()
    
    # Categories (no dependencies)
    deleted_counts['Categories'] = session.query(Category).delete()
    
    session.commit()
    
    print("🧹 Test Data Purged:")
    print("=" * 50)
    for table, count in deleted_counts.items():
        if count > 0:
            print(f"  {table:<20} {count} records deleted")
    print("=" * 50)
    print("✓ Database is now empty and ready for migration!")
    
except Exception as e:
    session.rollback()
    print(f"✗ Error purging test data: {e}")
    raise
finally:
    session.close()

2025-12-02 14:45:55,335 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-02 14:45:55,336 INFO sqlalchemy.engine.Engine DELETE FROM sku_features
2025-12-02 14:45:55,338 INFO sqlalchemy.engine.Engine [generated in 0.00122s] ()
2025-12-02 14:45:55,339 INFO sqlalchemy.engine.Engine DELETE FROM skus
2025-12-02 14:45:55,339 INFO sqlalchemy.engine.Engine [generated in 0.00037s] ()
2025-12-02 14:45:55,340 INFO sqlalchemy.engine.Engine DELETE FROM product_features
2025-12-02 14:45:55,341 INFO sqlalchemy.engine.Engine [generated in 0.00040s] ()
2025-12-02 14:45:55,341 INFO sqlalchemy.engine.Engine DELETE FROM features
2025-12-02 14:45:55,341 INFO sqlalchemy.engine.Engine [generated in 0.00028s] ()
2025-12-02 14:45:55,342 INFO sqlalchemy.engine.Engine DELETE FROM feature_types
2025-12-02 14:45:55,342 INFO sqlalchemy.engine.Engine [generated in 0.00038s] ()
2025-12-02 14:45:55,343 INFO sqlalchemy.engine.Engine DELETE FROM products
2025-12-02 14:45:55,343 INFO sqlalchemy.engine.Engine [genera

### Step 2: Migrate Categories

Extract unique categories and subcategories from the catalog.

In [10]:
session = Session()

try:
    # Get unique category combinations from Product_Catalog
    category_combos = catalog_tables['Product_Catalog'][['Main_Category', 'Sub_Category']].drop_duplicates()
    
    print(f"Found {len(category_combos)} unique category combinations:")
    
    for _, row in category_combos.iterrows():
        # Check if category already exists
        existing = session.query(Category).filter_by(
            category=row['Main_Category'],
            subcategory=row['Sub_Category']
        ).first()
        
        if not existing:
            category = Category(
                category=row['Main_Category'],
                subcategory=row['Sub_Category']
            )
            session.add(category)
            print(f"  + {row['Main_Category']} → {row['Sub_Category']}")
        else:
            print(f"  - {row['Main_Category']} → {row['Sub_Category']} (already exists)")
    
    session.commit()
    print("\n✓ Categories migrated successfully!")
    
except Exception as e:
    session.rollback()
    print(f"✗ Error migrating categories: {e}")
    raise
finally:
    session.close()

Found 11 unique category combinations:
2025-12-02 14:46:39,902 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-02 14:46:39,903 INFO sqlalchemy.engine.Engine SELECT categories.id AS categories_id, categories.category AS categories_category, categories.subcategory AS categories_subcategory, categories.created_at AS categories_created_at, categories.updated_at AS categories_updated_at 
FROM categories 
WHERE categories.category = ? AND categories.subcategory = ?
 LIMIT ? OFFSET ?
2025-12-02 14:46:39,903 INFO sqlalchemy.engine.Engine [generated in 0.00051s] ('Shirt', 'Tee Shirt', 1, 0)
  + Shirt → Tee Shirt
2025-12-02 14:46:39,905 INFO sqlalchemy.engine.Engine INSERT INTO categories (category, subcategory, created_at, updated_at) VALUES (?, ?, ?, ?)
2025-12-02 14:46:39,905 INFO sqlalchemy.engine.Engine [cached since 278.6s ago] ('Shirt', 'Tee Shirt', '2025-12-02 20:46:39.905319', '2025-12-02 20:46:39.905323')
2025-12-02 14:46:39,906 INFO sqlalchemy.engine.Engine SELECT categories.id

### Step 3: Migrate Feature Types

Create feature types from the code sheet names.

In [11]:
session = Session()

try:
    # Get feature sheet names (exclude Product_Catalog, Prefix_Codes, Main_Category, Sub_Category)
    exclude_sheets = ['Product_Catalog', 'Prefix_Codes', 'Main_Category', 'Sub_Category']
    feature_sheets = [name for name in catalog_tables.keys() if name not in exclude_sheets]
    
    print(f"Creating {len(feature_sheets)} feature types:")
    
    for idx, feature_name in enumerate(feature_sheets):
        existing = session.query(FeatureType).filter_by(feature_type=feature_name).first()
        
        if not existing:
            feature_type = FeatureType(
                feature_type=feature_name,
                display_order=idx + 1,
                is_required=True  # You can adjust this per feature type
            )
            session.add(feature_type)
            print(f"  + {feature_name} (order: {idx + 1})")
        else:
            print(f"  - {feature_name} (already exists)")
    
    session.commit()
    print("\n✓ Feature types migrated successfully!")
    
except Exception as e:
    session.rollback()
    print(f"✗ Error migrating feature types: {e}")
    raise
finally:
    session.close()

Creating 6 feature types:
2025-12-02 14:47:49,825 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-02 14:47:49,827 INFO sqlalchemy.engine.Engine SELECT feature_types.id AS feature_types_id, feature_types.feature_type AS feature_types_feature_type, feature_types.display_order AS feature_types_display_order, feature_types.is_required AS feature_types_is_required, feature_types.created_at AS feature_types_created_at 
FROM feature_types 
WHERE feature_types.feature_type = ?
 LIMIT ? OFFSET ?
2025-12-02 14:47:49,827 INFO sqlalchemy.engine.Engine [generated in 0.00064s] ('Size', 1, 0)
  + Size (order: 1)
2025-12-02 14:47:49,829 INFO sqlalchemy.engine.Engine INSERT INTO feature_types (feature_type, display_order, is_required, created_at) VALUES (?, ?, ?, ?)
2025-12-02 14:47:49,829 INFO sqlalchemy.engine.Engine [generated in 0.00038s] ('Size', 1, 1, '2025-12-02 20:47:49.829032')
2025-12-02 14:47:49,830 INFO sqlalchemy.engine.Engine SELECT feature_types.id AS feature_types_id, feature_typ

### Step 4: Migrate Features

Load all feature values from the code sheets.

In [12]:
session = Session()

try:
    exclude_sheets = ['Product_Catalog', 'Prefix_Codes', 'Main_Category', 'Sub_Category']
    feature_sheets = [name for name in catalog_tables.keys() if name not in exclude_sheets]
    
    total_features = 0
    
    for feature_type_name in feature_sheets:
        # Get the feature type from DB
        feature_type = session.query(FeatureType).filter_by(feature_type=feature_type_name).first()
        
        if not feature_type:
            print(f"✗ Feature type '{feature_type_name}' not found, skipping...")
            continue
        
        # Get the feature data from Numbers
        feature_df = catalog_tables[feature_type_name]
        
        print(f"\n{feature_type_name}:")
        
        for idx, row in feature_df.iterrows():
            # Check if feature already exists
            existing = session.query(Feature).filter_by(
                feature_type_id=feature_type.id,
                code=row['Prefix']
            ).first()
            
            if not existing:
                feature = Feature(
                    feature_type_id=feature_type.id,
                    code=row['Prefix'],
                    name=row['Name'],
                    description=row.get('Comments', None),
                    sort_order=int(idx),
                    is_active=True
                )
                session.add(feature)
                total_features += 1
                print(f"  + {row['Prefix']}: {row['Name']}")
            else:
                print(f"  - {row['Prefix']}: {row['Name']} (already exists)")
    
    session.commit()
    print(f"\n✓ {total_features} features migrated successfully!")
    
except Exception as e:
    session.rollback()
    print(f"✗ Error migrating features: {e}")
    raise
finally:
    session.close()

2025-12-02 14:47:52,786 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-02 14:47:52,787 INFO sqlalchemy.engine.Engine SELECT feature_types.id AS feature_types_id, feature_types.feature_type AS feature_types_feature_type, feature_types.display_order AS feature_types_display_order, feature_types.is_required AS feature_types_is_required, feature_types.created_at AS feature_types_created_at 
FROM feature_types 
WHERE feature_types.feature_type = ?
 LIMIT ? OFFSET ?
2025-12-02 14:47:52,787 INFO sqlalchemy.engine.Engine [cached since 2.961s ago] ('Size', 1, 0)

Size:
2025-12-02 14:47:52,790 INFO sqlalchemy.engine.Engine SELECT features.id AS features_id, features.feature_type_id AS features_feature_type_id, features.code AS features_code, features.name AS features_name, features.description AS features_description, features.sort_order AS features_sort_order, features.is_active AS features_is_active, features.created_at AS features_created_at, features.updated_at AS features_updated_at

### Step 5: Migrate Products

Load all products from the Product_Catalog.

In [13]:
session = Session()

try:
    products_added = 0
    
    for idx, row in catalog_tables['Product_Catalog'].iterrows():
        # Find the category
        category = session.query(Category).filter_by(
            category=row['Main_Category'],
            subcategory=row['Sub_Category']
        ).first()
        
        if not category:
            print(f"✗ Category not found for {row['Name']}, skipping...")
            continue
        
        # Check if product already exists
        existing = session.query(Product).filter_by(product_name=row['Name']).first()
        
        if not existing:
            product = Product(
                product_name=row['Name'],
                category_id=category.id,
                description=None,  # Add if you have description field
                base_price=None,   # Add if you have price field
                status='active'
            )
            session.add(product)
            products_added += 1
            print(f"  + {row['Name']}")
        else:
            print(f"  - {row['Name']} (already exists)")
    
    session.commit()
    print(f"\n✓ {products_added} products migrated successfully!")
    
except Exception as e:
    session.rollback()
    print(f"✗ Error migrating products: {e}")
    raise
finally:
    session.close()

2025-12-02 14:47:57,627 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-02 14:47:57,628 INFO sqlalchemy.engine.Engine SELECT categories.id AS categories_id, categories.category AS categories_category, categories.subcategory AS categories_subcategory, categories.created_at AS categories_created_at, categories.updated_at AS categories_updated_at 
FROM categories 
WHERE categories.category = ? AND categories.subcategory = ?
 LIMIT ? OFFSET ?
2025-12-02 14:47:57,629 INFO sqlalchemy.engine.Engine [cached since 77.73s ago] ('Shirt', 'Tee Shirt', 1, 0)
2025-12-02 14:47:57,630 INFO sqlalchemy.engine.Engine SELECT products.id AS products_id, products.product_name AS products_product_name, products.category_id AS products_category_id, products.description AS products_description, products.base_price AS products_base_price, products.status AS products_status, products.created_at AS products_created_at, products.updated_at AS products_updated_at 
FROM products 
WHERE products.product_name =

### Step 6: Link Products to Features

Create Product_Features relationships based on the catalog.

In [14]:
session = Session()

try:
    links_added = 0
    exclude_sheets = ['Product_Catalog', 'Prefix_Codes', 'Main_Category', 'Sub_Category']
    feature_columns = [col for col in catalog_tables['Product_Catalog'].columns 
                      if col not in ['Index', 'Main_Category', 'Sub_Category', 'Name']]
    
    for idx, row in catalog_tables['Product_Catalog'].iterrows():
        # Get the product
        product = session.query(Product).filter_by(product_name=row['Name']).first()
        
        if not product:
            continue
        
        # For each feature column, parse the values and create links
        for feature_col in feature_columns:
            feature_values_str = str(row[feature_col])
            
            # Skip empty or None values
            if pd.isna(row[feature_col]) or feature_values_str == 'nan':
                continue
            
            # Parse comma-separated values
            feature_value_names = [v.strip() for v in feature_values_str.split(',')]
            
            # Get the feature type
            feature_type = session.query(FeatureType).filter_by(feature_type=feature_col).first()
            
            if not feature_type:
                continue
            
            # Link each feature value to the product
            for feature_name in feature_value_names:
                # Find the feature by name and type
                feature = session.query(Feature).join(FeatureType).filter(
                    Feature.name == feature_name,
                    FeatureType.id == feature_type.id
                ).first()
                
                if feature:
                    # Check if link already exists
                    existing_link = session.query(ProductFeature).filter_by(
                        product_id=product.id,
                        feature_id=feature.id
                    ).first()
                    
                    if not existing_link:
                        link = ProductFeature(
                            product_id=product.id,
                            feature_id=feature.id
                        )
                        session.add(link)
                        links_added += 1
    
    session.commit()
    print(f"✓ {links_added} product-feature links created!")
    
except Exception as e:
    session.rollback()
    print(f"✗ Error creating product-feature links: {e}")
    raise
finally:
    session.close()

2025-12-02 14:48:02,422 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-02 14:48:02,423 INFO sqlalchemy.engine.Engine SELECT products.id AS products_id, products.product_name AS products_product_name, products.category_id AS products_category_id, products.description AS products_description, products.base_price AS products_base_price, products.status AS products_status, products.created_at AS products_created_at, products.updated_at AS products_updated_at 
FROM products 
WHERE products.product_name = ?
 LIMIT ? OFFSET ?
2025-12-02 14:48:02,424 INFO sqlalchemy.engine.Engine [cached since 4.794s ago] ('Bandit Banquet Tee', 1, 0)
2025-12-02 14:48:02,425 INFO sqlalchemy.engine.Engine SELECT feature_types.id AS feature_types_id, feature_types.feature_type AS feature_types_feature_type, feature_types.display_order AS feature_types_display_order, feature_types.is_required AS feature_types_is_required, feature_types.created_at AS feature_types_created_at 
FROM feature_types 
WHERE featu

### Step 7: Verify Migration

Query the database to confirm all data was migrated correctly.

In [15]:
session = Session()

try:
    print("DATABASE MIGRATION SUMMARY")
    print("=" * 80)
    
    # Count records in each table
    categories_count = session.query(Category).count()
    feature_types_count = session.query(FeatureType).count()
    features_count = session.query(Feature).count()
    products_count = session.query(Product).count()
    product_features_count = session.query(ProductFeature).count()
    
    print(f"\n📊 Record Counts:")
    print(f"  Categories:       {categories_count}")
    print(f"  Feature Types:    {feature_types_count}")
    print(f"  Features:         {features_count}")
    print(f"  Products:         {products_count}")
    print(f"  Product-Features: {product_features_count}")
    
    # Show sample product with its features
    print(f"\n📦 Sample Product:")
    sample_product = session.query(Product).first()
    if sample_product:
        print(f"  Name: {sample_product.product_name}")
        print(f"  Category: {sample_product.category.category} → {sample_product.category.subcategory}")
        print(f"  Available Features:")
        
        for pf in sample_product.product_features:
            feature = pf.feature
            print(f"    - {feature.feature_type.feature_type}: {feature.name} ({feature.code})")
    
    print("\n" + "=" * 80)
    print("✓ Migration verification complete!")
    
except Exception as e:
    print(f"✗ Error verifying migration: {e}")
    raise
finally:
    session.close()

DATABASE MIGRATION SUMMARY
2025-12-02 14:48:11,516 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-02 14:48:11,520 INFO sqlalchemy.engine.Engine SELECT count(*) AS count_1 
FROM (SELECT categories.id AS categories_id, categories.category AS categories_category, categories.subcategory AS categories_subcategory, categories.created_at AS categories_created_at, categories.updated_at AS categories_updated_at 
FROM categories) AS anon_1
2025-12-02 14:48:11,521 INFO sqlalchemy.engine.Engine [generated in 0.00084s] ()
2025-12-02 14:48:11,522 INFO sqlalchemy.engine.Engine SELECT count(*) AS count_1 
FROM (SELECT feature_types.id AS feature_types_id, feature_types.feature_type AS feature_types_feature_type, feature_types.display_order AS feature_types_display_order, feature_types.is_required AS feature_types_is_required, feature_types.created_at AS feature_types_created_at 
FROM feature_types) AS anon_1
2025-12-02 14:48:11,523 INFO sqlalchemy.engine.Engine [generated in 0.00032s] ()
2025-

### Table 1: Categories

**Purpose**: Store product categories and subcategories

| Column | Type | Constraints | Description |
|--------|------|-------------|-------------|
| id | INTEGER | PRIMARY KEY, AUTOINCREMENT | Unique identifier |
| category | VARCHAR(100) | NOT NULL | Main category (e.g., "Apparel") |
| subcategory | VARCHAR(100) | | Subcategory (e.g., "T-Shirts") |
| created_at | TIMESTAMP | DEFAULT CURRENT_TIMESTAMP | Record creation time |
| updated_at | TIMESTAMP | DEFAULT CURRENT_TIMESTAMP | Last update time |

**Indexes**: 
- Index on (category, subcategory) for lookup performance

**Notes**: 
- Add any additional fields you need here

### Additional Tables to Consider

**What else do you need to track?**

Ideas:
- **Suppliers**: Vendor information
- **Images**: Product/SKU images (URLs or paths)
- **Pricing History**: Track price changes over time
- **Inventory Transactions**: Log inventory changes
- **Sales Channels**: Where products are sold (website, store, etc.)
- **Product Relationships**: Bundles, variants, related products
- **Custom Fields**: Flexible metadata storage

**Tell me what additional columns or tables you want to add!**

### Table 7: SKU Features

**Purpose**: Link each SKU to its specific feature combination

| Column | Type | Constraints | Description |
|--------|------|-------------|-------------|
| id | INTEGER | PRIMARY KEY, AUTOINCREMENT | Unique identifier |
| sku_id | INTEGER | FOREIGN KEY → skus.id | The SKU |
| feature_id | INTEGER | FOREIGN KEY → features.id | Feature in this SKU |
| created_at | TIMESTAMP | DEFAULT CURRENT_TIMESTAMP | Record creation time |

**Indexes**: 
- Index on sku_id
- Index on feature_id

**Constraints**:
- UNIQUE(sku_id, feature_id) - no duplicate features per SKU

**Notes**: 
- This allows querying all SKUs with a specific feature (e.g., "all Black products")

### Table 6: SKUs

**Purpose**: Store generated SKUs with all their feature combinations

| Column | Type | Constraints | Description |
|--------|------|-------------|-------------|
| id | INTEGER | PRIMARY KEY, AUTOINCREMENT | Unique identifier |
| sku | VARCHAR(100) | NOT NULL, UNIQUE | Generated SKU code |
| product_id | INTEGER | FOREIGN KEY → products.id | Base product |
| batch_name | VARCHAR(200) | | Batch identifier (for tracking) |
| price | DECIMAL(10,2) | | SKU-specific price (if different from base) |
| inventory_qty | INTEGER | DEFAULT 0 | Current inventory count |
| status | VARCHAR(20) | DEFAULT 'active' | active/inactive/discontinued |
| generated_at | TIMESTAMP | DEFAULT CURRENT_TIMESTAMP | When SKU was generated |
| updated_at | TIMESTAMP | DEFAULT CURRENT_TIMESTAMP | Last update time |

**Indexes**: 
- Index on sku (already implied by UNIQUE)
- Index on product_id
- Index on batch_name
- Index on status

**Notes**: 
- What other SKU-level fields do you need? (weight, dimensions, UPC, etc.?)

### Table 5: Product Features

**Purpose**: Link products to their available features (which colors/sizes apply to which products)

| Column | Type | Constraints | Description |
|--------|------|-------------|-------------|
| id | INTEGER | PRIMARY KEY, AUTOINCREMENT | Unique identifier |
| product_id | INTEGER | FOREIGN KEY → products.id | Product this applies to |
| feature_id | INTEGER | FOREIGN KEY → features.id | Available feature |
| created_at | TIMESTAMP | DEFAULT CURRENT_TIMESTAMP | Record creation time |

**Indexes**: 
- Index on product_id
- Index on feature_id

**Constraints**:
- UNIQUE(product_id, feature_id) - no duplicate feature assignments

**Notes**: 
- This junction table defines which features are valid for each product

### Table 4: Features

**Purpose**: Store individual feature values (specific colors, sizes, etc.)

| Column | Type | Constraints | Description |
|--------|------|-------------|-------------|
| id | INTEGER | PRIMARY KEY, AUTOINCREMENT | Unique identifier |
| feature_type_id | INTEGER | FOREIGN KEY → feature_types.id | Type of feature |
| code | VARCHAR(20) | NOT NULL | Short code for SKU (e.g., "BLK", "XL") |
| name | VARCHAR(100) | NOT NULL | Display name (e.g., "Black", "Extra Large") |
| description | TEXT | | Additional details |
| sort_order | INTEGER | | Order within feature type |
| is_active | BOOLEAN | DEFAULT TRUE | Whether feature is currently available |
| created_at | TIMESTAMP | DEFAULT CURRENT_TIMESTAMP | Record creation time |
| updated_at | TIMESTAMP | DEFAULT CURRENT_TIMESTAMP | Last update time |

**Indexes**: 
- Index on (feature_type_id, code)
- Index on is_active

**Constraints**:
- UNIQUE(feature_type_id, code) - no duplicate codes within same feature type

**Notes**: 
- Any additional feature attributes you want to track?

### Table 3: Feature Types

**Purpose**: Define types of features (Color, Size, Design, etc.)

| Column | Type | Constraints | Description |
|--------|------|-------------|-------------|
| id | INTEGER | PRIMARY KEY, AUTOINCREMENT | Unique identifier |
| feature_type | VARCHAR(50) | NOT NULL, UNIQUE | e.g., "Color", "Size", "Design" |
| display_order | INTEGER | | Order for display purposes |
| is_required | BOOLEAN | DEFAULT FALSE | Whether feature is required for SKU |
| created_at | TIMESTAMP | DEFAULT CURRENT_TIMESTAMP | Record creation time |

**Notes**: 
- This allows us to add new feature types without schema changes

### Table 2: Products

**Purpose**: Store base product models (before permutations)

| Column | Type | Constraints | Description |
|--------|------|-------------|-------------|
| id | INTEGER | PRIMARY KEY, AUTOINCREMENT | Unique identifier |
| product_name | VARCHAR(200) | NOT NULL, UNIQUE | Product model name |
| category_id | INTEGER | FOREIGN KEY → categories.id | Link to category |
| description | TEXT | | Product description |
| base_price | DECIMAL(10,2) | | Base price (optional) |
| status | VARCHAR(20) | DEFAULT 'active' | active/inactive/discontinued |
| created_at | TIMESTAMP | DEFAULT CURRENT_TIMESTAMP | Record creation time |
| updated_at | TIMESTAMP | DEFAULT CURRENT_TIMESTAMP | Last update time |

**Indexes**: 
- Index on product_name
- Index on category_id
- Index on status

**Notes**: 
- What other product-level fields do you need?

### Draft Schema in Mermaid

**See the rendered ER diagram in `docs/schema-diagram.md`**

Open that file and press `Cmd+Shift+V` to view the Mermaid diagram with all table relationships.

## Phase 2: Data Migration

## Phase 3: SKU Generation Logic

## Phase 4: PostgreSQL Preparation